In [19]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
df = pd.read_csv('filtered100_10k.csv')
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,18212874291345894768,0,14102317,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,20108,320,50,2299,2,1319,-1,52
1,795643102997507814,0,14102900,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,22261,320,50,2545,0,167,-1,221
2,11559228699423998953,0,14102616,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15707,320,50,1722,0,35,-1,79
3,14414547350492900822,1,14102816,1005,1,e8f79e60,c4342784,f028772b,ecad2386,7801e8d9,...,1,0,23405,320,50,2682,1,419,-1,212
4,8440814477931021634,0,14102607,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,-1,79


In [21]:
df.shape

(10000, 24)

In [22]:
df.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [23]:
import datetime

def isWeekend(d):
    """
    take df['hour'] as input and check whether the day is in the weekend.
    """
    d = str(d)
    year = '20' + d[:2]
    month = d[2:4]
    day = d[4:6]
    return int(datetime.date(int(year), int(month), int(day)).isoweekday()>4)

def hour(d):
    """
    take df['hour'] as input and convert it to:
        1 (morning 6-12) or,
        2 (afternoon 13-18) or,
        3 (night 19-5)
    """
    hour = int(str(d)[6:])
    if hour >=6 and hour <=12:
        return 1
    elif hour >= 13 and hour <= 18:
        return 2
    else:
        return 3
    
df = df.assign(day=pd.Series(df['hour'].apply(isWeekend)).values)
df = df.assign(hour=pd.Series(df['hour'].apply(hour)).values)
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,day
0,18212874291345894768,0,2,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,20108,320,50,2299,2,1319,-1,52,0
1,795643102997507814,0,3,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,22261,320,50,2545,0,167,-1,221,0
2,11559228699423998953,0,2,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,15707,320,50,1722,0,35,-1,79,1
3,14414547350492900822,1,2,1005,1,e8f79e60,c4342784,f028772b,ecad2386,7801e8d9,...,0,23405,320,50,2682,1,419,-1,212,0
4,8440814477931021634,0,1,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,15706,320,50,1722,0,35,-1,79,1


In [24]:
for col in df.columns:
    print(col + " " + str(len(df[col].unique())))

id 10000
click 2
hour 3
C1 4
banner_pos 3
site_id 284
site_domain 224
site_category 11
app_id 146
app_domain 21
app_category 10
device_id 656
device_ip 44
device_model 603
device_type 4
device_conn_type 3
C14 521
C15 4
C16 5
C17 168
C18 4
C19 39
C20 12
C21 39
day 2


In [25]:
n = df['device_ip'].value_counts()[df['device_ip'].value_counts()>=50].index
df = df.loc[df['device_ip'].isin(n)]
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,day
0,18212874291345894768,0,2,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,20108,320,50,2299,2,1319,-1,52,0
1,795643102997507814,0,3,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,22261,320,50,2545,0,167,-1,221,0
2,11559228699423998953,0,2,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,15707,320,50,1722,0,35,-1,79,1
3,14414547350492900822,1,2,1005,1,e8f79e60,c4342784,f028772b,ecad2386,7801e8d9,...,0,23405,320,50,2682,1,419,-1,212,0
4,8440814477931021634,0,1,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,0,15706,320,50,1722,0,35,-1,79,1


In [26]:
df.shape

(10000, 25)

In [27]:
for col in df.columns:
    print(col + " " + str(len(df[col].unique())))

id 10000
click 2
hour 3
C1 4
banner_pos 3
site_id 284
site_domain 224
site_category 11
app_id 146
app_domain 21
app_category 10
device_id 656
device_ip 44
device_model 603
device_type 4
device_conn_type 3
C14 521
C15 4
C16 5
C17 168
C18 4
C19 39
C20 12
C21 39
day 2


In [28]:
df = df.drop('id', 1) # remove id
df.head()

,click,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,20108,320,50,2299,2,1319,-1,52
1,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,22261,320,50,2545,0,167,-1,221
2,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,15707,320,50,1722,0,35,-1,79
3,1,1005,1,e8f79e60,c4342784,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,23405,320,50,2682,1,419,-1,212
4,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,15706,320,50,1722,0,35,-1,79


# Feature hashing

We use feature hashing: https://github.com/scikit-learn-contrib/categorical-encoding/blob/master/category_encoders/hashing.py

In [29]:
import category_encoders as ce

retain = 50
y = df['click']
col = ['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', \
      'app_category', 'device_id', 'device_model', 'device_type', 'device_conn_type', \
      'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
enc = ce.HashingEncoder(cols=col, n_components=retain).fit(df, y)
df = enc.transform(df)
#print(df.info())
df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,click,device_ip
0,2,0,0,0,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,fffe2401
1,3,0,0,1,0,0,1,0,1,1,...,1,0,0,0,0,0,1,0,0,fffe2401
2,3,0,0,0,0,0,1,0,1,1,...,1,0,0,0,1,0,0,0,0,fffe2401
3,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,2,0,1,1,fffe2401
4,3,0,0,0,0,0,1,0,1,1,...,1,0,1,0,1,0,0,0,0,fffe2401


In [30]:
for col in df.columns:
    print(col + " " + str(len(df[col].unique())))

col_0 7
col_1 4
col_2 4
col_3 4
col_4 5
col_5 4
col_6 4
col_7 3
col_8 4
col_9 4
col_10 4
col_11 5
col_12 4
col_13 3
col_14 4
col_15 4
col_16 3
col_17 3
col_18 4
col_19 5
col_20 3
col_21 3
col_22 3
col_23 4
col_24 4
col_25 3
col_26 3
col_27 3
col_28 4
col_29 4
col_30 4
col_31 4
col_32 4
col_33 4
col_34 3
col_35 3
col_36 6
col_37 3
col_38 4
col_39 3
col_40 4
col_41 3
col_42 4
col_43 4
col_44 3
col_45 3
col_46 3
col_47 3
col_48 4
col_49 3
click 2
device_ip 44


In [31]:
grouped = df.groupby(['device_ip'])
k = 9
l = []
for name, group in grouped:
    user_interactions = group.groupby('click') # there will be 2 groups: 0/1
    try:
        ones = user_interactions.get_group(1)
    except:
        print('no ones')
        continue
    zeros = user_interactions.get_group(0)
    # compute the number of splits per user and split the interactions
    num_of_splits = len(zeros)//k
    splits = np.array_split(zeros, num_of_splits)
    # build the rounds with k-zeros rewards a 1-one reward
    for index_and_row, split in zip(ones.iterrows(), splits):
        r = pd.concat([split, index_and_row[1].to_frame().transpose()])
        # add the round to the list (after shuffling)
        if len(r)>10:
            r.drop(r.index[:len(r)-10], inplace=True)
        l.append(r.iloc[np.random.permutation(len(r))])

no ones


In [32]:
import random
random.shuffle(l)

In [33]:
for i in range(10):
    print(l[i])

     col_0 col_1 col_2 col_3 col_4 col_5 col_6 col_7 col_8 col_9    ...     \
5270     3     0     1     0     0     0     1     0     0     0    ...      
5269     3     0     1     0     0     0     1     0     0     0    ...      
5265     3     0     1     0     0     0     1     0     0     0    ...      
5274     3     0     1     0     0     0     1     0     0     0    ...      
5273     3     0     1     0     0     0     1     0     0     0    ...      
5277     3     0     1     0     0     0     1     0     0     0    ...      
5276     3     0     1     0     0     0     1     0     0     0    ...      
5267     3     0     1     0     0     0     1     0     0     0    ...      
5266     3     0     1     0     0     0     1     0     0     0    ...      
4987     3     0     1     0     0     0     1     0     0     0    ...      

     col_42 col_43 col_44 col_45 col_46 col_47 col_48 col_49 click device_ip  
5270      1      1      0      0      0      0      0      0  

In [34]:
df = pd.concat(l)
rewards = pd.DataFrame(df['click'])
users = pd.DataFrame(df['device_ip'])

col = [item for item in df.columns if item not in ['click', 'device_ip']]
print(col)
#col = ['col_' + str(i) for i in range(retain)]
df = pd.concat([pd.get_dummies(df[c]) for c in col], axis=1)
df = df.div(df.sum(axis=1), axis=0) # normalize
df.head()

['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14', 'col_15', 'col_16', 'col_17', 'col_18', 'col_19', 'col_20', 'col_21', 'col_22', 'col_23', 'col_24', 'col_25', 'col_26', 'col_27', 'col_28', 'col_29', 'col_30', 'col_31', 'col_32', 'col_33', 'col_34', 'col_35', 'col_36', 'col_37', 'col_38', 'col_39', 'col_40', 'col_41', 'col_42', 'col_43', 'col_44', 'col_45', 'col_46', 'col_47', 'col_48', 'col_49']


,0,1,2,3,4,5,6,0,1,2,...,0,1,2,0,1,2,3,0,1,2
5270,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5269,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5265,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5274,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5273,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0


In [35]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(list(users['device_ip']))
users['device_ip'] = le.transform(users['device_ip'])
sorted(users['device_ip'].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42]

In [36]:
rewards.to_csv('processed/medium/reward_list.csv')
df.to_csv('processed/medium/processed10k.csv')
users.to_csv('processed/medium/users.csv')

In [37]:
t = 0
df.iloc[t*10:t*10+10]

,0,1,2,3,4,5,6,0,1,2,...,0,1,2,0,1,2,3,0,1,2
5270,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5269,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5265,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5274,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5273,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5277,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5276,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5267,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
5266,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
4987,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0,...,0.02,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.0,0.0
